In [4]:
import dgl
import dgl.nn as gnn
import dgl.function as fn

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

Для задачи предсказания связи нужен negative sampling. Negative sampling реализован в `EdgeDataLoader` из коробки. Чтобы воспользоваться им, нужно указать функцию негативного сэмплирования, например, `dgl.dataloading.negative_sampler.Uniform(k)`, которая для каждого существующего ребра генерирует `k` отрицательных примеров. Можно реализовать и свои функции сэмплинга.

Для каждого батча `DGL` при создании негативных примеров генерирует 3 сущности:
1. Граф, содержащий все ребра из минибатча (positive graph)
2. Граф, содержащий несуществующие ребра, полученные при негативном сэмплировании (negative graph)
2. Список MFGs



In [2]:
# Пример: кастомный сэмплер, который генерирует ребра, выбирая конечные узлы
# пропорционально их степени

class NegativeSampler:
    def __init__(self, G, k, gamma=.75):
        self.weights = G.in_degrees().float() ** gamma
        self.k = k

    def __call__(self, G, eids):
        src, _ = G.find_edges(eids)
        src = src.repeat_interleave(self.k)
        dts = self.weights.multinomial(len(src), replacement=True)
        return src, dst

In [19]:
# копия из 10_dgl_stochastic_node_classification
class GCN(nn.Module):
    def __init__(self, n_inputs, n_hidden, n_outputs):
        super().__init__()
        self.conv1 = gnn.SAGEConv(n_inputs, n_hidden, aggregator_type='mean', activation=F.relu)
        self.conv2 = gnn.SAGEConv(n_hidden, n_outputs, aggregator_type='mean')

    def forward(self, blocks, features):
        assert len(blocks) == 2
        out = self.conv1(blocks[0], features)
        out = self.conv2(blocks[1], out)
        return out

class ScorePredictor(nn.Module):
    def forward(self, edge_subgraph, features):
        with edge_subgraph.local_scope():
            edge_subgraph.ndata['h'] = features
            edge_subgraph.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            return edge_subgraph.edata['score']

class Model(nn.Module):
    def __init__(self, n_inputs, n_hidden, n_outputs):
        super().__init__()
        self.conv = GCN(n_inputs, n_hidden, n_outputs)
        self.predictor = ScorePredictor()

    def forward(self, positive_graph, negative_graph, blocks, features):
        out = self.conv(blocks, features)
        pos_score = self.predictor(positive_graph, out)
        neg_score = self.predictor(negative_graph, out)
        return pos_score, neg_score

def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones_like(pos_score), torch.zeros_like(neg_score)])
    return F.binary_cross_entropy_with_logits(scores, labels)

In [21]:
dataset = dgl.data.CoraGraphDataset()
G = dataset[0]

n_inputs = G.ndata['feat'].shape[1]
n_hidden = 16
n_epochs = 1000

train_eids = torch.arange(G.num_edges())

sampler = dgl.dataloading.MultiLayerFullNeighborSampler(n_layers=2)
dataloader = dgl.dataloading.EdgeDataLoader(G, train_eids, sampler,
                                            negative_sampler=dgl.dataloading.negative_sampler.Uniform(5),
                                            batch_size=256,
                                            shuffle=True)


model = Model(n_inputs, n_hidden, n_hidden)
optimizer = optim.Adam(model.parameters(), lr=.001)

for epoch in range(10):
    for step, (input_nodes, pG, nG, blocks) in enumerate(dataloader):
        # forward
        batch_features = blocks[0].srcdata['feat']
        pos_score, neg_score = model(pG, nG, blocks, batch_features)
        loss = compute_loss(pos_score, neg_score)
        # backward
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if not step % 20:
            print('Epoch {:05d} | Step {:05d} | Loss {:.4f}'.format(epoch, step, loss.item()))

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Epoch 00000 | Step 00000 | Loss 0.6936
